In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import os

In [2]:
#This is the general approch to getting data for our model
#(1) randomly generate a year between 1996-2020
#(2) From this year, select at random a S&P 500 from that year 
#(3) With this stock, measure the expected return from the year to year+1 and also 
#    measure actual return of year+1 to year+2 (this is our binary classification data)
#   get the relevant earings/revenue data from that year 
#(4) From this year, meaure the percent change in CPI and Money Supply
#(5) Repeat this for n number of samples
#(6) Create the data matrix 

In [3]:
years_considered = [i for i in range(1998, 2021)]

In [4]:
#cell that gets one sample
year = np.random.choice(years_considered)
day = np.random.choice([i for i in range(1,27)])
month = np.random.choice([i for i in range(1, 12)])
date = str(year) + str(month) + str(day)
datestring = str(year) + '-' + str(month) + '-' + str(day)
print(date)

def get_table(filename):
    if os.path.isfile(filename):
        df = pd.read_csv(filename, index_col='date')
        return df

#takes in date string an returns a stock ticker
def get_company(date):
    #load in the historical list of companies
    filename = './Data/S&P 500 Historical Components & Changes(03-14-2022).csv'
    df = get_table(filename)
    # Convert ticker column from csv to list, then sort.
    df['tickers'] = df['tickers'].apply(lambda x: sorted(x.split(',')))
    df2 = df[df.index <= date]
    last_row = df2.tail(1)
    member_list = last_row['tickers'][0]
    return np.random.choice(member_list)

ticker = get_company(datestring)
print("Ticker: {}".format(ticker))

def get_return(symbol, start, end):
    # create empty dataframe and list to store data frames
    df = pd.DataFrame()
    data = []
    # print the symbol which is being downloaded
    try:
        stock = []
        stock = yf.download(symbol, start=start, end=end, progress=False)
        if len(stock) == 0:
            return None
        df = stock
        df.drop(columns=['Open', 'Low', 'High', 'Close', 'Volume'], inplace=True)
        new_df = df['Adj Close'].pct_change()
        m = new_df.mean()
        # convert to annual expected returns
        # annual = (dialy_return + 1)^365 - 1
        annual = ((m + 1) ** 365) - 1
    except:
        return None
    return annual

#binary classifier
def get_class(symbol, start, end, threshold=.07):
    df = pd.DataFrame()
    stock = []
    stock = yf.download(symbol, start=start, end=end, progress=False)
    if len(stock) == 0:
        return None
    df = stock
    df.drop(columns=['Open', 'Low', 'High', 'Close', 'Volume'], inplace=True)
    ret = (df['Adj Close'][-1] - df['Adj Close'][0]) / df['Adj Close'][0]
    print(ret)
    if ret > threshold: return 1
    else: return 0
    

exp_ret = get_return(ticker, datetime.datetime(year,month,day), datetime.datetime(year+1,month,day))
#don't append if None
if exp_ret != None:
    print(exp_ret)
    clas = get_class(ticker, datetime.datetime(year+1,month,day), datetime.datetime(year+2,month,day))
    print(clas)

200054
Ticker: UTX

1 Failed download:
- UTX: No data found, symbol may be delisted


In [6]:
def get_M2(start, end, filename ='./Data/FRB_H6.csv'):
    df = pd.read_csv(filename)
    print(df)
    df.tail()

cpi = get_M2(str(year) + '-' + str(month) + '-' + str(day),str(year+1) + '-' + str(month) + '-' + str(day))

      Series Description M1; Not seasonally adjusted  \
0                  Unit:                    Currency   
1            Multiplier:                       1e+09   
2              Currency:                         USD   
3    Unique Identifier:              H6/H6_M1/M1_N.M   
4            Time Period                      M1_N.M   
..                   ...                         ...   
758              2021-10                     20022.4   
759              2021-11                     20271.1   
760              2021-12                     20600.1   
761              2022-01                     20568.1   
762              2022-02                     20560.8   

    M2; Not seasonally adjusted Currency; Not seasonally adjusted  \
0                      Currency                          Currency   
1                         1e+09                             1e+09   
2                           USD                               USD   
3               H6/H6_M2/M2_N.M                  H6

In [ ]:


#function to generate samples
def generate_samples(years=years_considered, num_samples=500):
    
    
    